<a href="https://colab.research.google.com/github/ytyimin/scm518/blob/main/Machine_Replacement_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equipment Replacement: Purchasing, Selling, and Maintainance

## Objective and Prerequisites

This equipment replacement problem shows you how to use the shortest path approach to find the optimal planning of purchasing and maintainance of an equipment over multiple time periods. The key strategy is to use nodes to represent different quarters and then build valid arcs that connects these nodes to represent the duration of keeping the machine before replacing. For each valid arc,the associated cost can be computed by incorporating purchasing cost, cumulative maintainance cost and the final salvage value before replacing. The objectives of the problem are:

* Identify optimal equipment replacment schedule over the five year period,
* Minimize the total purchasing and maintainance cost while taking into consideration the salvage value, and
* Ensure that the replacement schedule is valid, i.e., cannot replace the equipment less than one year and cannot keep the equipment more than three years.

This modeling example is at the introductory level, where we assume that you know Python and that you have some knowledge of how to build mathematical optimization models.

---
## Problem Description

![picture](https://drive.google.com/uc?id=12shoDxA9pQcIVbEpIFBIiyzwpDpw4q4b)

VanBuren Metals is a manufacturing company that uses many large machines to work on metals. These machines require frequent maintenance beacuse of wear and tear, and VanBuren finds that it is sometimes advantageous, from a cost standpoint, to replace machines rather than continue to maintain them. For one particular class of machines, the company has estimated the quarterly costs of maintenance, the salvage value from reselling an old machine, and the cost to purchase a new machine. 

The maintenance cost and the salvage value depend on the age of the current machine (at the beginning of the quarter). However, the maintenance costs, the salvage values, and the purchase cost do not depend on time. In other words, there is no inflation. 

* The purchase cost of a new machine is always \$3,530.
* The maintenance cost of a machine in its first quarter of use is \$100. For each succeeding quarter, the maintenance cost increases by \$65. This reflects the fact that machines require more maintenance as they age.
* The salvage value of a machine after one quarter of use is \$1,530. After each succeeding quarter of use, the salvage value decreases by \$110.

VanBuren wants to devise a strategy for purchasing machines over the next five years. As a matter of policy, the ompany never sells a machines that is less than one year old, and it never keeps a machine that is more than three years old. Also, the machine in use at the beginning of the current quarter is brand new.

This example shows how to use a shortest path representation to solve a problem is that not obvious to be a network flow problem.

## Model Formulation

### Indices

$i,j,k \in \{1..21\}$: Index of nodes representing quarters.

### Parameters

$A$: Set of tuples ($i,j$) where keeping machine from quarter (node) $i$ to quarter (node) $j$ is feasible.

$p$: Purchase cost of a new machine. [$p$=$3,530].

$s_0$: Initial salvage value. [$s_0$=$1,530].

$s_1$: Depreciation per quarter (loss in salvage value). [$s_1$=$110].

$m_0$: Initial maintainance cost. [$m_0$=$100].

$m_1$: Maginal increase in maintainance cost after each quarter. [$m_1$=$65].


$c_{ij}$: Aggregate cost of keeping machine from quarter $i$ to quarter $j$, $(i,j) \in A$. Note that we can directly compute $c_{ij}$ as
\begin{equation}
c_{ij} = p+ \sum_{k=0}^{j-i-1}(m_0+m_1*k) -(s_0-s_1*(j-i-1))
\end{equation}


### Decision Variables

$x_{ij}$: Whether to keep machine from quarter $i$ to quarter $j$, $(i,j) \in A$.


### Objective Function

- **Cost**. We want to minimize the aggregate operating cost.


\begin{equation}
\text{Min}_{x_{ij}} \quad \sum_{(i,j) \in A} c_{ij}*x_{ij}
\tag{0}
\end{equation}

### Constraints

\begin{equation}
\sum_{i|(i,j) \in A} x_{ij} - \sum_{k|(j,k) \in A} x_{jk} \geq 1 \quad \forall j \in \{21\} \quad (\text{planning horizaon up to quarter 21})
\tag{1}
\end{equation}

\begin{equation}
\sum_{j|(i,j) \in A} x_{ij} - \sum_{k|(k,i) \in A} x_{ki} \leq 1 \quad \forall i \in \{1\} \quad (\text{start planning horizon in quarter 1})
\tag{2}
\end{equation}

\begin{equation}
\sum_{i|(i,j) \in A} x_{ij} - \sum_{k|(j,k) \in A} x_{jk} =0 \quad \forall j \in \{5,...,20\} \quad (\text{continual operation of machine})
\tag{3}
\end{equation}

\begin{equation}
x_{ij} \in \{0,1\} \quad \forall (i,j) \in A \quad (\text{binary buying/selling decision})
\tag{4}
\end{equation}

---

## Python Implementation

We now import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 5.4 MB/s 


In [ ]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

Set up the model and solve

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('equipment maintainance')

# indices for companies and routes
node = [*range(0,21)]
start = [*range(0,1)]
transit = [*range(1,20)]
end = [*range(20,21)]

#print(node)
#print(start)
#print(transit)
#print(end)

# various cost/price

p = 3530
s0 = 1530
s1 = 110
m0 = 100
m1 = 65

# build c_ij matrix

c = [[0 for i in node] for j in node]

# Valid set of tuples
A = []
for i in node:
    for j in node:
        if j-i >= 4 and j-i <= 11:
            
            mc = 0
            for k in range(0,j-i):
              mc += m0+m1*k

            c[i][j] = p + mc - (s0-s1*(j-i-1))

            tp = i,j
            A.append(tp)

print(np.matrix(c))

# Valid set of tuples
#A = []
#for i in node:
#    for j in node:
#        if c[i][j] > 0:
#            tp = i,j
#            A.append(tp)

# take a look at the set
# print(np.matrix(A))

# valid set of inbound routes for node j
AI = [] 
k = 0
for l in node:
    A_temp = []
    for i in node:
        for j in node:
            if c[i][j] > 0:
                if j==k:
                    tp = i,j
                    A_temp.append(tp)
    AI.append(A_temp) 
    k+=1               

# take a look at a sample
# print(np.matrix(AI[0]))

# valid set of outbound routes for node j
AO = [] 
k = 0
for l in node:
    A_temp = []
    for i in node:
        for j in node:
            if c[i][j] > 0:
                if i==k:
                    tp = i,j
                    A_temp.append(tp)
    AO.append(A_temp) 
    k+=1               

# take a look at a sample
# print(np.matrix(AO[0]))

# Build decision variables: where to assign company i to route j
x = m.addVars(A, vtype=GRB.BINARY, name='Keep')
    
# Objective function: Minimize total payroll cost
m.setObjective(gp.quicksum(c[i][j]*x[(i,j)] for i,j in A), GRB.MINIMIZE)
    
# Reach the fifth year (21 quarter)
endConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[j]) - gp.quicksum(x[(j,k)] for j,k in AO[j]) >= 1 for j in end), 
                                      name='endConstrs')

# Cannot exceed plant capacity
startConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AO[i]) - gp.quicksum(x[(k,i)] for k,i in AI[i]) <= 1 for i in start), 
                                      name='startConstrs')

# Blanacing for transit nodes
transitConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[j]) - gp.quicksum(x[(j,k)] for j,k in AO[j]) == 0 for j in transit), 
                                      name='transitConstrs')

# Run optimization engine
m.optimize()

Restricted license - for non-production use only - expires 2023-10-25
[[   0    0    0    0 3120 3590 4125 4725 5390 6120 6915 7775    0    0
     0    0    0    0    0    0    0]
 [   0    0    0    0    0 3120 3590 4125 4725 5390 6120 6915 7775    0
     0    0    0    0    0    0    0]
 [   0    0    0    0    0    0 3120 3590 4125 4725 5390 6120 6915 7775
     0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0 3120 3590 4125 4725 5390 6120 6915
  7775    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0 3120 3590 4125 4725 5390 6120
  6915 7775    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0 3120 3590 4125 4725 5390
  6120 6915 7775    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0 3120 3590 4125 4725
  5390 6120 6915 7775    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0 3120 3590 4125
  4725 5390 6120 6915 7775    0    0]
 [   0    0    0    0    0    0    0    0    0    

Take a look at the results of shipments

In [ ]:
#####################################################
#         Shipment results
#####################################################
    
print(f"\n\n___Optimal shipment from plants to customers________")
t_cost = 0
for i,j in A:
    if x[(i,j)].x > 0:
        if i<=0:
          s_node_type = "start"
        elif i<=19:
          s_node_type = "transit"
        else:
          s_node_type = "end"
        if j<=0:
          d_node_type = "start"
        elif j<=19:
          d_node_type = "transit"
        else:
          d_node_type = "end"
             
        print("Keep machines from %s quarter %2d to %s quarter %2d" % (s_node_type, i+1, d_node_type, j+1))
        t_cost += x[(i,j)].x*c[i][j]

print("The total cost of equipment maintainance over the five years is $%5d" % (t_cost))
            



___Optimal shipment from plants to customers________
Keep machines from start quarter  1 to transit quarter  8
Keep machines from transit quarter  8 to transit quarter 15
Keep machines from transit quarter 15 to end quarter 21
The total cost of equipment maintainance over the five years is $13575


As we can see, all customer demands are satisfied with the utilization of warehouses. The plant capacity are not exceeded. Furthermore, the optimal solution uses transhipment strategy, for example, shipping from customer 6 to customer 7. 

---
##  Conclusion

In this example, we addressed the tomato shipping problem. We determined the optimal shipment  of tomatos from plants to customers: 
* Satisfy demand for each customer, 
* Minimize the total shipping cost,  
* Ensure plant capacities are not exceeded, and
* Utilize transhipment to reduce shipping cost.

A special technique in the model formulation is sparse reprentation, where we significantly reduce the number of decision variables by restricting the set of decisions to be on the valid routes only. This benefit becomes more significant as problem size grows.

This tomato shipment model can be used in many different settings to help companies make informed decisions about satisfying customer demands from a set of plants where there are transit stations allowing for transhipments.


##  References
[1] Sixty examples of business optimization models. https://ytyimin.github.io/tart-cherry/.

[2] Gurobi python reference. https://www.gurobi.com/documentation/

[3] This note book is developed by Yimin Wang. If you have any comments or suggestions, please contact yimin_wang@asu.edu